# Chapter 32: TypeScript with React

React is the most widely used frontend library for building user interfaces. When combined with TypeScript, it provides exceptional developer experience through compile-time type checking, intelligent autocomplete, and safer refactoring. This chapter covers everything you need to know about using TypeScript effectively with React, from basic setup to advanced patterns.

---

## 32.1 Setting Up React with TypeScript

There are several ways to create a React project with TypeScript support. Modern tooling makes this process straightforward with zero configuration required.

### Using Vite (Recommended)

Vite is the fastest and most modern build tool for React applications.

```bash
# Create a new React + TypeScript project with Vite
npm create vite@latest my-react-app -- --template react-ts

# Navigate and install dependencies
cd my-react-app
npm install
npm run dev
```

**Project Structure:**
```
my-react-app/
├── src/
│   ├── assets/          # Static assets
│   ├── App.tsx          # Root component
│   ├── main.tsx         # Entry point
│   └── vite-env.d.ts    # Vite type declarations
├── index.html
├── tsconfig.json        # TypeScript configuration
├── tsconfig.node.json   # Node-specific TS config
└── vite.config.ts       # Vite configuration with TypeScript
```

**vite.config.ts:**
```typescript
import { defineConfig } from 'vite'
import react from '@vitejs/plugin-react'

// https://vitejs.dev/config/
export default defineConfig({
  plugins: [react()],
  // TypeScript checking during build
  build: {
    target: 'ES2020',
    outDir: 'dist',
  },
})
```

### Using Create React App (Legacy)

While still supported, CRA is now in maintenance mode. Vite or Next.js are recommended for new projects.

```bash
npx create-react-app my-app --template typescript
```

### Using Next.js

For production applications requiring SSR/SSG:

```bash
npx create-next-app@latest my-next-app --typescript
```

**Key Configuration Files:**

**tsconfig.json** (Vite/React standards):
```json
{
  "compilerOptions": {
    "target": "ES2020",
    "useDefineForClassFields": true,
    "lib": ["ES2020", "DOM", "DOM.Iterable"],
    "module": "ESNext",
    "skipLibCheck": true,
    "moduleResolution": "bundler",
    "allowImportingTsExtensions": true,
    "resolveJsonModule": true,
    "isolatedModules": true,
    "noEmit": true,
    "jsx": "react-jsx",
    "strict": true,
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noFallthroughCasesInSwitch": true
  },
  "include": ["src"],
  "references": [{ "path": "./tsconfig.node.json" }]
}
```

**Explanation:**
- `jsx: "react-jsx"`: Uses the new JSX transform (no need to import React)
- `moduleResolution: "bundler"`: Modern resolution strategy compatible with Vite/Webpack 5
- `noEmit: true`: Bundler handles compilation, TypeScript only checks types
- `strict: true`: Enables all strict type-checking options

---

## 32.2 Typing Component Props

Props are the primary mechanism for passing data and callbacks to React components. TypeScript makes props explicit and self-documenting.

### Basic Props with Interfaces

```typescript
// Define props interface
interface UserCardProps {
  name: string;
  email: string;
  age: number;
  isVerified: boolean;
}

// Component using props
function UserCard({ name, email, age, isVerified }: UserCardProps) {
  return (
    <div className="user-card">
      <h2>{name} {isVerified && '✓'}</h2>
      <p>Email: {email}</p>
      <p>Age: {age}</p>
    </div>
  );
}

// Usage
function App() {
  return (
    <UserCard 
      name="John Doe"
      email="john@example.com"
      age={30}
      isVerified={true}
    />
  );
}
```

**Explanation:**
- Interface clearly documents what props the component expects
- TypeScript checks that all required props are provided
- Incorrect types (e.g., `age="30"`) are caught at compile time
- Destructuring in parameters makes the code cleaner

### Optional Props and Defaults

```typescript
interface ButtonProps {
  label: string;
  variant?: 'primary' | 'secondary' | 'danger';  // Optional with union type
  size?: 'small' | 'medium' | 'large';
  disabled?: boolean;
  onClick?: () => void;
}

function Button({ 
  label, 
  variant = 'primary', 
  size = 'medium', 
  disabled = false,
  onClick 
}: ButtonProps) {
  return (
    <button 
      className={`btn btn-${variant} btn-${size}`}
      disabled={disabled}
      onClick={onClick}
    >
      {label}
    </button>
  );
}

// Usage - all optional props can be omitted
<Button label="Click me" />
<Button label="Danger" variant="danger" size="large" />
<Button label="Disabled" disabled onClick={() => console.log('clicked')} />
```

### Typing Children

The `children` prop requires special attention as it can be various types.

```typescript
import { ReactNode, ReactElement } from 'react';

// ReactNode - most flexible, accepts almost anything
interface ContainerProps {
  children: ReactNode;  // string, number, element, array, fragment, etc.
  className?: string;
}

// ReactElement - only React elements (components)
interface StrictContainerProps {
  children: ReactElement;
}

// Specific children types
interface ListProps {
  children: ReactElement<ListItemProps>[];  // Array of ListItem components
}

interface CardProps {
  header: ReactNode;
  body: ReactNode;
  footer?: ReactNode;
}

function Card({ header, body, footer }: CardProps) {
  return (
    <div className="card">
      <div className="card-header">{header}</div>
      <div className="card-body">{body}</div>
      {footer && <div className="card-footer">{footer}</div>}
    </div>
  );
}

// Usage
<Card 
  header={<h2>Title</h2>}
  body={<p>Content here</p>}
  footer={<button>Action</button>}
/>
```

**ReactNode vs ReactElement:**
- `ReactNode`: Broadest type, includes `string`, `number`, `null`, `undefined`, arrays, etc.
- `ReactElement`: Only JSX elements (result of component rendering)
- `ReactChild`: Elements or strings/numbers (excludes arrays)
- Use `ReactNode` for most `children` props as it's the most flexible

### Component Props with Generics

For reusable components that work with different data types:

```typescript
interface ListProps<T> {
  items: T[];
  renderItem: (item: T, index: number) => ReactNode;
  keyExtractor: (item: T, index: number) => string | number;
  className?: string;
}

// Generic component function
function List<T>({ items, renderItem, keyExtractor, className }: ListProps<T>) {
  return (
    <ul className={className}>
      {items.map((item, index) => (
        <li key={keyExtractor(item, index)}>
          {renderItem(item, index)}
        </li>
      ))}
    </ul>
  );
}

// Usage with User type
interface User {
  id: number;
  name: string;
  email: string;
}

function UserList({ users }: { users: User[] }) {
  return (
    <List<User>
      items={users}
      renderItem={(user) => (
        <div>
          <strong>{user.name}</strong>
          <span>{user.email}</span>
        </div>
      )}
      keyExtractor={(user) => user.id}
      className="user-list"
    />
  );
}

// Usage with Product type
interface Product {
  sku: string;
  title: string;
  price: number;
}

function ProductList({ products }: { products: Product[] }) {
  return (
    <List<Product>
      items={products}
      renderItem={(product) => (
        <div>
          <span>{product.title}</span>
          <span>${product.price}</span>
        </div>
      )}
      keyExtractor={(product) => product.sku}
    />
  );
}
```

**Explanation:**
- `<T>` makes the component generic
- TypeScript infers `T` from the `items` prop
- `renderItem` receives correctly typed item automatically
- Provides type safety while maintaining reusability

---

## 32.3 Typing Component State

State management is core to React. TypeScript ensures state updates are type-safe and helps prevent common bugs.

### useState with Type Inference

```typescript
import { useState } from 'react';

function Counter() {
  // TypeScript infers: number
  const [count, setCount] = useState(0);
  
  // TypeScript infers: string
  const [name, setName] = useState('');
  
  // TypeScript infers: boolean
  const [isActive, setIsActive] = useState(false);

  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
      
      <input 
        value={name}
        onChange={(e) => setName(e.target.value)}
      />
      
      <button onClick={() => setIsActive(!isActive)}>
        {isActive ? 'Deactivate' : 'Activate'}
      </button>
    </div>
  );
}
```

### useState with Explicit Types

When initial value is `null` or undefined, or when state can be multiple types:

```typescript
interface User {
  id: number;
  name: string;
  email: string;
}

function UserProfile() {
  // User | null - starts as null, becomes User after loading
  const [user, setUser] = useState<User | null>(null);
  const [loading, setLoading] = useState<boolean>(true);
  const [error, setError] = useState<string | null>(null);

  useEffect(() => {
    fetchUser()
      .then(data => {
        setUser(data);
        setLoading(false);
      })
      .catch(err => {
        setError(err.message);
        setLoading(false);
      });
  }, []);

  if (loading) return <div>Loading...</div>;
  if (error) return <div>Error: {error}</div>;
  if (!user) return <div>No user found</div>;

  // TypeScript knows user is User here, not null
  return (
    <div>
      <h1>{user.name}</h1>
      <p>{user.email}</p>
    </div>
  );
}
```

### Complex State Objects

```typescript
interface FormState {
  values: {
    email: string;
    password: string;
    confirmPassword: string;
  };
  errors: {
    email?: string;
    password?: string;
    confirmPassword?: string;
  };
  touched: {
    email: boolean;
    password: boolean;
    confirmPassword: boolean;
  };
  isSubmitting: boolean;
}

function RegistrationForm() {
  const [formState, setFormState] = useState<FormState>({
    values: {
      email: '',
      password: '',
      confirmPassword: ''
    },
    errors: {},
    touched: {
      email: false,
      password: false,
      confirmPassword: false
    },
    isSubmitting: false
  });

  const updateField = (field: keyof FormState['values'], value: string) => {
    setFormState(prev => ({
      ...prev,
      values: {
        ...prev.values,
        [field]: value
      },
      touched: {
        ...prev.touched,
        [field]: true
      }
    }));
  };

  return (
    <form>
      <input
        type="email"
        value={formState.values.email}
        onChange={(e) => updateField('email', e.target.value)}
      />
      {formState.touched.email && formState.errors.email && (
        <span className="error">{formState.errors.email}</span>
      )}
      {/* Other fields... */}
    </form>
  );
}
```

### useReducer for Complex State Logic

For complex state logic, `useReducer` provides better type safety than `useState`:

```typescript
import { useReducer } from 'react';

// State type
interface State {
  count: number;
  step: number;
}

// Action types using discriminated union
type Action =
  | { type: 'increment' }
  | { type: 'decrement' }
  | { type: 'reset'; payload: number }
  | { type: 'setStep'; payload: number };

// Reducer function with explicit types
function reducer(state: State, action: Action): State {
  switch (action.type) {
    case 'increment':
      return { ...state, count: state.count + state.step };
    case 'decrement':
      return { ...state, count: state.count - state.step };
    case 'reset':
      return { ...state, count: action.payload };
    case 'setStep':
      return { ...state, step: action.payload };
    default:
      // Exhaustive check ensures all actions are handled
      const _exhaustiveCheck: never = action;
      return state;
  }
}

function Counter() {
  const [state, dispatch] = useReducer(reducer, { count: 0, step: 1 });

  return (
    <div>
      <p>Count: {state.count}</p>
      <button onClick={() => dispatch({ type: 'decrement' })}>-</button>
      <button onClick={() => dispatch({ type: 'increment' })}>+</button>
      <button onClick={() => dispatch({ type: 'reset', payload: 0 })}>
        Reset
      </button>
      <input
        type="number"
        value={state.step}
        onChange={(e) => 
          dispatch({ type: 'setStep', payload: Number(e.target.value) })
        }
      />
    </div>
  );
}
```

**Explanation:**
- Discriminated union `Action` ensures type safety in reducer
- TypeScript checks that `action.payload` exists for actions that require it
- `never` type in default case ensures all action types are handled
- State shape is explicitly defined and enforced

---

## 32.4 Functional Components and Hooks

Modern React uses functional components with Hooks. TypeScript enhances these with type safety.

### Typing Functional Components

```typescript
import { FC, ReactElement, ReactNode } from 'react';

// Method 1: FC (FunctionComponent) - includes children by default
const Card: FC<{ title: string }> = ({ title, children }) => {
  return (
    <div className="card">
      <h2>{title}</h2>
      <div>{children}</div>
    </div>
  );
};

// Method 2: Direct annotation (Recommended)
// More explicit, better performance, children not automatically included
interface Props {
  title: string;
  children?: ReactNode;
}

function CardBetter({ title, children }: Props): ReactElement {
  return (
    <div className="card">
      <h2>{title}</h2>
      <div>{children}</div>
    </div>
  );
}

// Method 3: With generic props
interface PropsWithGeneric<T> {
  data: T;
  render: (item: T) => ReactNode;
}

function GenericComponent<T>({ data, render }: PropsWithGeneric<T>): ReactElement {
  return <div>{render(data)}</div>;
}
```

**Best Practice:** Method 2 (direct annotation) is generally preferred in modern React because:
- `FC` includes `children` implicitly which may not be desired
- `FC` adds overhead and can cause issues with generic components
- Direct annotation is clearer about what the component actually accepts

### Typing useEffect

```typescript
import { useEffect } from 'react';

function UserProfile({ userId }: { userId: number }) {
  const [user, setUser] = useState<User | null>(null);

  // Effect with cleanup function
  useEffect(() => {
    let isMounted = true;  // Flag to prevent state updates on unmounted component
    
    async function fetchUser() {
      try {
        const data = await api.getUser(userId);
        if (isMounted) {
          setUser(data);
        }
      } catch (error) {
        if (isMounted) {
          console.error(error);
        }
      }
    }

    fetchUser();

    // Cleanup function
    return () => {
      isMounted = false;
    };
  }, [userId]);  // Dependency array type-checked

  // Effect with timer
  useEffect(() => {
    const timer = setInterval(() => {
      console.log('Tick');
    }, 1000);

    // Cleanup clears interval
    return () => clearInterval(timer);
  }, []);

  return <div>{user?.name}</div>;
}
```

### Typing useCallback and useMemo

```typescript
import { useCallback, useMemo } from 'react';

interface Props {
  items: Item[];
  onSelect: (item: Item) => void;
  filterText: string;
}

function ItemList({ items, onSelect, filterText }: Props) {
  // useMemo with typed return
  const filteredItems = useMemo<Item[]>(() => {
    return items.filter(item => 
      item.name.toLowerCase().includes(filterText.toLowerCase())
    );
  }, [items, filterText]);

  // useCallback with typed function
  const handleClick = useCallback((item: Item) => {
    console.log('Clicked:', item.id);
    onSelect(item);
  }, [onSelect]);

  return (
    <ul>
      {filteredItems.map(item => (
        <ListItem 
          key={item.id} 
          item={item} 
          onClick={handleClick}
        />
      ))}
    </ul>
  );
}

interface Item {
  id: number;
  name: string;
}

interface ListItemProps {
  item: Item;
  onClick: (item: Item) => void;
}

const ListItem = memo(function ListItem({ item, onClick }: ListItemProps) {
  return (
    <li onClick={() => onClick(item)}>
      {item.name}
    </li>
  );
});
```

**Explanation:**
- `useMemo<ReturnType>` ensures the memoized value has correct type
- `useCallback` preserves the function type across renders
- `memo` with component functions requires proper typing of props
- Dependency arrays are type-checked to ensure all dependencies are included

### Typing useRef

`useRef` has two main use cases: DOM references and mutable values.

```typescript
import { useRef, useEffect } from 'react';

function TextInputWithFocusButton() {
  // DOM element ref - initialized as null
  const inputRef = useRef<HTMLInputElement>(null);

  useEffect(() => {
    // TypeScript knows current is HTMLInputElement | null
    inputRef.current?.focus();
  }, []);

  return <input ref={inputRef} type="text" />;
}

// Mutable value ref (not for DOM)
function Timer() {
  const intervalRef = useRef<number | null>(null);
  const countRef = useRef(0);  // Inferred as number

  const start = () => {
    if (intervalRef.current) return;
    
    intervalRef.current = window.setInterval(() => {
      countRef.current += 1;
      console.log(countRef.current);
    }, 1000);
  };

  const stop = () => {
    if (intervalRef.current) {
      clearInterval(intervalRef.current);
      intervalRef.current = null;
    }
  };

  return (
    <div>
      <button onClick={start}>Start</button>
      <button onClick={stop}>Stop</button>
    </div>
  );
}

// Forwarding refs with forwardRef
import { forwardRef, useImperativeHandle } from 'react';

interface FancyInputProps {
  placeholder?: string;
  onChange?: (value: string) => void;
}

// Exposed methods via ref
export interface FancyInputHandle {
  focus: () => void;
  clear: () => void;
  getValue: () => string;
}

export const FancyInput = forwardRef<FancyInputHandle, FancyInputProps>(
  function FancyInput({ placeholder, onChange }, ref) {
    const inputRef = useRef<HTMLInputElement>(null);

    useImperativeHandle(ref, () => ({
      focus: () => inputRef.current?.focus(),
      clear: () => {
        if (inputRef.current) {
          inputRef.current.value = '';
        }
      },
      getValue: () => inputRef.current?.value || ''
    }), []);

    return (
      <input
        ref={inputRef}
        type="text"
        placeholder={placeholder}
        onChange={(e) => onChange?.(e.target.value)}
      />
    );
  }
);

// Usage
function Parent() {
  const fancyInputRef = useRef<FancyInputHandle>(null);

  const handleClick = () => {
    fancyInputRef.current?.focus();
    console.log(fancyInputRef.current?.getValue());
  };

  return (
    <>
      <FancyInput ref={fancyInputRef} placeholder="Enter text" />
      <button onClick={handleClick}>Focus and Log</button>
    </>
  );
}
```

---

## 32.5 Event Handling Types

React events are synthetic events that wrap native browser events. TypeScript provides specific types for these.

### Form Events

```typescript
import { FormEvent, ChangeEvent } from 'react';

interface FormData {
  email: string;
  password: string;
  rememberMe: boolean;
}

function LoginForm() {
  const [formData, setFormData] = useState<FormData>({
    email: '',
    password: '',
    rememberMe: false
  });

  // Form submission event
  const handleSubmit = (event: FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    console.log('Submitting:', formData);
  };

  // Input change events
  const handleChange = (event: ChangeEvent<HTMLInputElement>) => {
    const { name, value, type, checked } = event.target;
    
    setFormData(prev => ({
      ...prev,
      [name]: type === 'checkbox' ? checked : value
    }));
  };

  return (
    <form onSubmit={handleSubmit}>
      <input
        type="email"
        name="email"
        value={formData.email}
        onChange={handleChange}
        placeholder="Email"
      />
      <input
        type="password"
        name="password"
        value={formData.password}
        onChange={handleChange}
        placeholder="Password"
      />
      <label>
        <input
          type="checkbox"
          name="rememberMe"
          checked={formData.rememberMe}
          onChange={handleChange}
        />
        Remember me
      </label>
      <button type="submit">Login</button>
    </form>
  );
}
```

### Mouse and Keyboard Events

```typescript
import { MouseEvent, KeyboardEvent, DragEvent, ClipboardEvent } from 'react';

function EventExamples() {
  // Click event with currentTarget vs target
  const handleClick = (event: MouseEvent<HTMLButtonElement>) => {
    // event.currentTarget is the button element (typed as HTMLButtonElement)
    console.log('Button clicked:', event.currentTarget.textContent);
    
    // event.target could be any child element
    console.log('Actual target:', (event.target as HTMLElement).tagName);
  };

  // Keyboard events
  const handleKeyDown = (event: KeyboardEvent<HTMLInputElement>) => {
    if (event.key === 'Enter') {
      console.log('Enter pressed');
    }
    if (event.ctrlKey && event.key === 's') {
      event.preventDefault();  // Prevent save dialog
      console.log('Ctrl+S detected');
    }
  };

  // Drag and drop
  const handleDragStart = (event: DragEvent<HTMLDivElement>) => {
    event.dataTransfer.setData('text/plain', 'dragged-item');
    event.dataTransfer.effectAllowed = 'move';
  };

  const handleDrop = (event: DragEvent<HTMLDivElement>) => {
    event.preventDefault();
    const data = event.dataTransfer.getData('text/plain');
    console.log('Dropped:', data);
  };

  // Clipboard events
  const handlePaste = (event: ClipboardEvent<HTMLInputElement>) => {
    const pastedText = event.clipboardData.getData('text');
    console.log('Pasted:', pastedText);
    
    // Prevent pasting if it contains numbers
    if (/\d/.test(pastedText)) {
      event.preventDefault();
    }
  };

  return (
    <div>
      <button onClick={handleClick}>Click me</button>
      
      <input 
        onKeyDown={handleKeyDown}
        onPaste={handlePaste}
        placeholder="Type here..."
      />
      
      <div 
        draggable
        onDragStart={handleDragStart}
        onDrop={handleDrop}
        onDragOver={(e) => e.preventDefault()}
      >
        Drag me
      </div>
    </div>
  );
}
```

### Custom Event Handlers

When passing event handlers as props:

```typescript
// Define handler types explicitly
interface ButtonProps {
  onClick: (event: MouseEvent<HTMLButtonElement>) => void;
  onDoubleClick?: (event: MouseEvent<HTMLButtonElement>) => void;
}

function CustomButton({ onClick, onDoubleClick }: ButtonProps) {
  return (
    <button 
      onClick={onClick}
      onDoubleClick={onDoubleClick}
    >
      Click me
    </button>
  );
}

// Usage
function Parent() {
  const handleClick = (e: MouseEvent<HTMLButtonElement>) => {
    console.log('Clicked at:', e.clientX, e.clientY);
  };

  return <CustomButton onClick={handleClick} />;
}
```

---

## 32.6 Typing Context

Context provides a way to pass data through the component tree without prop drilling. TypeScript ensures type safety when consuming context.

### Creating Type-Safe Context

```typescript
import { createContext, useContext, useState, ReactNode } from 'react';

// Define the shape of context data
interface Theme {
  name: 'light' | 'dark';
  colors: {
    primary: string;
    background: string;
    text: string;
  };
}

interface ThemeContextType {
  theme: Theme;
  toggleTheme: () => void;
  setTheme: (theme: Theme) => void;
}

// Create context with undefined as initial value
const ThemeContext = createContext<ThemeContextType | undefined>(undefined);

// Provider component
interface ThemeProviderProps {
  children: ReactNode;
  initialTheme?: Theme;
}

const defaultLightTheme: Theme = {
  name: 'light',
  colors: {
    primary: '#007bff',
    background: '#ffffff',
    text: '#000000'
  }
};

const defaultDarkTheme: Theme = {
  name: 'dark',
  colors: {
    primary: '#375a7f',
    background: '#222222',
    text: '#ffffff'
  }
};

export function ThemeProvider({ children, initialTheme }: ThemeProviderProps) {
  const [theme, setTheme] = useState<Theme>(initialTheme || defaultLightTheme);

  const toggleTheme = () => {
    setTheme(current => 
      current.name === 'light' ? defaultDarkTheme : defaultLightTheme
    );
  };

  const value: ThemeContextType = {
    theme,
    toggleTheme,
    setTheme
  };

  return (
    <ThemeContext.Provider value={value}>
      {children}
    </ThemeContext.Provider>
  );
}

// Custom hook for consuming context
export function useTheme(): ThemeContextType {
  const context = useContext(ThemeContext);
  
  if (context === undefined) {
    throw new Error('useTheme must be used within a ThemeProvider');
  }
  
  return context;
}

// Usage
function ThemedButton() {
  const { theme, toggleTheme } = useTheme();
  
  return (
    <button 
      onClick={toggleTheme}
      style={{ 
        backgroundColor: theme.colors.primary,
        color: theme.colors.text
      }}
    >
      Switch to {theme.name === 'light' ? 'dark' : 'light'} mode
    </button>
  );
}

function App() {
  return (
    <ThemeProvider>
      <ThemedButton />
    </ThemeProvider>
  );
}
```

**Explanation:**
- Context is created with `ThemeContextType | undefined` to force checking
- Custom hook `useTheme` throws error if used outside provider
- This pattern ensures type safety and prevents undefined errors
- State and updater functions are fully typed

### Context with Reducer Pattern

For complex state, combine Context with useReducer:

```typescript
import { createContext, useContext, useReducer, ReactNode } from 'react';

// State
interface AuthState {
  user: User | null;
  isAuthenticated: boolean;
  isLoading: boolean;
  error: string | null;
}

interface User {
  id: string;
  email: string;
  name: string;
}

// Actions
type AuthAction =
  | { type: 'AUTH_START' }
  | { type: 'AUTH_SUCCESS'; payload: User }
  | { type: 'AUTH_FAILURE'; payload: string }
  | { type: 'LOGOUT' }
  | { type: 'CLEAR_ERROR' };

// Context type
interface AuthContextType extends AuthState {
  login: (email: string, password: string) => Promise<void>;
  logout: () => void;
  clearError: () => void;
}

const AuthContext = createContext<AuthContextType | undefined>(undefined);

// Reducer
function authReducer(state: AuthState, action: AuthAction): AuthState {
  switch (action.type) {
    case 'AUTH_START':
      return { ...state, isLoading: true, error: null };
    case 'AUTH_SUCCESS':
      return { 
        ...state, 
        isLoading: false, 
        isAuthenticated: true, 
        user: action.payload 
      };
    case 'AUTH_FAILURE':
      return { 
        ...state, 
        isLoading: false, 
        isAuthenticated: false, 
        error: action.payload 
      };
    case 'LOGOUT':
      return { 
        ...state, 
        isAuthenticated: false, 
        user: null 
      };
    case 'CLEAR_ERROR':
      return { ...state, error: null };
    default:
      return state;
  }
}

// Provider
export function AuthProvider({ children }: { children: ReactNode }) {
  const [state, dispatch] = useReducer(authReducer, {
    user: null,
    isAuthenticated: false,
    isLoading: false,
    error: null
  });

  const login = async (email: string, password: string) => {
    dispatch({ type: 'AUTH_START' });
    try {
      // API call
      const user = await api.login(email, password);
      dispatch({ type: 'AUTH_SUCCESS', payload: user });
    } catch (error) {
      dispatch({ 
        type: 'AUTH_FAILURE', 
        payload: error instanceof Error ? error.message : 'Unknown error'
      });
    }
  };

  const logout = () => {
    dispatch({ type: 'LOGOUT' });
  };

  const clearError = () => {
    dispatch({ type: 'CLEAR_ERROR' });
  };

  const value: AuthContextType = {
    ...state,
    login,
    logout,
    clearError
  };

  return (
    <AuthContext.Provider value={value}>
      {children}
    </AuthContext.Provider>
  );
}

export function useAuth(): AuthContextType {
  const context = useContext(AuthContext);
  if (!context) throw new Error('useAuth must be used within AuthProvider');
  return context;
}
```

---

## 32.7 Typing Custom Hooks

Custom hooks extract component logic into reusable functions. TypeScript ensures these hooks are used correctly.

### Basic Custom Hook

```typescript
import { useState, useEffect, useCallback } from 'react';

// Hook for fetching data
interface UseFetchOptions {
  enabled?: boolean;
  refetchInterval?: number;
}

interface UseFetchResult<T> {
  data: T | null;
  isLoading: boolean;
  error: Error | null;
  refetch: () => void;
}

function useFetch<T>(
  url: string, 
  options: UseFetchOptions = {}
): UseFetchResult<T> {
  const { enabled = true, refetchInterval } = options;
  
  const [data, setData] = useState<T | null>(null);
  const [isLoading, setIsLoading] = useState(false);
  const [error, setError] = useState<Error | null>(null);

  const fetchData = useCallback(async () => {
    setIsLoading(true);
    setError(null);
    
    try {
      const response = await fetch(url);
      if (!response.ok) {
        throw new Error(`HTTP error! status: ${response.status}`);
      }
      const json = await response.json() as T;
      setData(json);
    } catch (err) {
      setError(err instanceof Error ? err : new Error('Unknown error'));
    } finally {
      setIsLoading(false);
    }
  }, [url]);

  useEffect(() => {
    if (!enabled) return;
    
    fetchData();
    
    if (refetchInterval) {
      const interval = setInterval(fetchData, refetchInterval);
      return () => clearInterval(interval);
    }
  }, [enabled, fetchData, refetchInterval]);

  return { data, isLoading, error, refetch: fetchData };
}

// Usage
interface User {
  id: number;
  name: string;
}

function UserProfile({ userId }: { userId: number }) {
  const { data: user, isLoading, error } = useFetch<User>(
    `/api/users/${userId}`,
    { enabled: userId > 0 }
  );

  if (isLoading) return <div>Loading...</div>;
  if (error) return <div>Error: {error.message}</div>;
  if (!user) return <div>No user</div>;

  return <div>{user.name}</div>;
}
```

### Generic Custom Hooks

```typescript
// Generic hook for localStorage
function useLocalStorage<T>(key: string, initialValue: T) {
  // Get stored value or initial value
  const [storedValue, setStoredValue] = useState<T>(() => {
    try {
      const item = window.localStorage.getItem(key);
      return item ? (JSON.parse(item) as T) : initialValue;
    } catch (error) {
      console.error(error);
      return initialValue;
    }
  });

  // Return a wrapped version of useState's setter function
  const setValue = useCallback((value: T | ((val: T) => T)) => {
    try {
      // Allow value to be a function so we have same API as useState
      const valueToStore = value instanceof Function ? value(storedValue) : value;
      setStoredValue(valueToStore);
      window.localStorage.setItem(key, JSON.stringify(valueToStore));
    } catch (error) {
      console.error(error);
    }
  }, [key, storedValue]);

  // Remove from localStorage
  const removeValue = useCallback(() => {
    try {
      window.localStorage.removeItem(key);
      setStoredValue(initialValue);
    } catch (error) {
      console.error(error);
    }
  }, [key, initialValue]);

  return [storedValue, setValue, removeValue] as const;
}

// Usage
function Settings() {
  const [theme, setTheme, resetTheme] = useLocalStorage<'light' | 'dark'>(
    'app-theme', 
    'light'
  );

  const [user, setUser, removeUser] = useLocalStorage<User | null>(
    'current-user',
    null
  );

  return (
    <div>
      <select 
        value={theme} 
        onChange={(e) => setTheme(e.target.value as 'light' | 'dark')}
      >
        <option value="light">Light</option>
        <option value="dark">Dark</option>
      </select>
      <button onClick={resetTheme}>Reset Theme</button>
      
      {user ? (
        <div>
          <p>Welcome, {user.name}</p>
          <button onClick={removeUser}>Logout</button>
        </div>
      ) : (
        <button onClick={() => setUser({ id: 1, name: 'John' })}>
          Login
        </button>
      )}
    </div>
  );
}
```

**Explanation:**
- `as const` assertion ensures the return type is a readonly tuple
- Generic `T` allows the hook to work with any type
- Function overload in `setValue` allows both direct values and updater functions
- Type safety ensures only valid theme values can be set

### Hook with Refs and Imperative API

```typescript
import { useRef, useState, useCallback } from 'react';

interface UseCounterResult {
  count: number;
  increment: () => void;
  decrement: () => void;
  reset: (value?: number) => void;
}

function useCounter(initialValue: number = 0): UseCounterResult {
  const [count, setCount] = useState<number>(initialValue);
  
  // Store initial value in ref for reset
  const initialValueRef = useRef(initialValue);

  const increment = useCallback(() => {
    setCount(c => c + 1);
  }, []);

  const decrement = useCallback(() => {
    setCount(c => c - 1);
  }, []);

  const reset = useCallback((value?: number) => {
    setCount(value ?? initialValueRef.current);
  }, []);

  return {
    count,
    increment,
    decrement,
    reset
  };
}

// Usage
function Counter() {
  const { count, increment, decrement, reset } = useCounter(10);

  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={decrement}>-</button>
      <button onClick={increment}>+</button>
      <button onClick={() => reset()}>Reset to 10</button>
      <button onClick={() => reset(0)}>Reset to 0</button>
    </div>
  );
}
```

---

## 32.8 Higher-Order Components with TypeScript

Higher-Order Components (HOCs) are functions that take a component and return a new component with additional props or behavior.

### Basic HOC Pattern

```typescript
import { ComponentType, ComponentPropsWithoutRef } from 'react';

// Props injected by the HOC
interface WithLoadingProps {
  isLoading: boolean;
}

// HOC function
function withLoading<T extends object>(
  WrappedComponent: ComponentType<T>
) {
  // Return new component with additional props
  return function WithLoadingComponent(
    props: T & WithLoadingProps
  ) {
    const { isLoading, ...rest } = props;
    
    if (isLoading) {
      return <div className="loading-spinner">Loading...</div>;
    }
    
    // Type assertion needed because we removed isLoading from props
    return <WrappedComponent {...rest as T} />;
  };
}

// Usage
interface UserProfileProps {
  user: {
    name: string;
    email: string;
  };
}

function UserProfile({ user }: UserProfileProps) {
  return (
    <div>
      <h1>{user.name}</h1>
      <p>{user.email}</p>
    </div>
  );
}

// Create wrapped component
const UserProfileWithLoading = withLoading(UserProfile);

// Usage in parent
function App() {
  const [isLoading, setIsLoading] = useState(true);
  const [user, setUser] = useState<UserProfileProps['user'] | null>(null);

  useEffect(() => {
    fetchUser().then(data => {
      setUser(data);
      setIsLoading(false);
    });
  }, []);

  return (
    <UserProfileWithLoading 
      isLoading={isLoading}
      user={user!}  // Non-null assertion or handle null case
    />
  );
}
```

### Advanced HOC with Display Name and Ref Forwarding

```typescript
import { 
  ComponentType, 
  forwardRef, 
  ForwardedRef, 
  useImperativeHandle,
  RefObject 
} from 'react';

// HOC with ref forwarding and display name
function withAuth<T extends object>(
  WrappedComponent: ComponentType<T>
) {
  // Define the component with ref forwarding
  function WithAuth(
    props: T,
    ref: ForwardedRef<HTMLDivElement>
  ) {
    const [isAuthenticated, setIsAuthenticated] = useState(false);

    useEffect(() => {
      checkAuth().then(setIsAuthenticated);
    }, []);

    if (!isAuthenticated) {
      return <div>Please log in</div>;
    }

    return <WrappedComponent {...props} ref={ref} />;
  }

  // Set display name for debugging
  const wrappedComponentName = WrappedComponent.displayName 
    || WrappedComponent.name 
    || 'Component';
  WithAuth.displayName = `withAuth(${wrappedComponentName})`;

  return forwardRef(WithAuth);
}

// HOC that injects additional props
interface WithThemeProps {
  theme: 'light' | 'dark';
  toggleTheme: () => void;
}

function withTheme<T extends object>(
  WrappedComponent: ComponentType<T & WithThemeProps>
) {
  return function WithThemeWrapper(props: Omit<T, keyof WithThemeProps>) {
    const [theme, setTheme] = useState<'light' | 'dark'>('light');
    
    const toggleTheme = () => {
      setTheme(t => t === 'light' ? 'dark' : 'light');
    };

    return (
      <WrappedComponent 
        {...props as T} 
        theme={theme} 
        toggleTheme={toggleTheme}
      />
    );
  };
}

// Compose multiple HOCs
interface BaseProps {
  title: string;
}

function BaseComponent({ title, theme, toggleTheme }: BaseProps & WithThemeProps) {
  return (
    <div style={{ background: theme === 'dark' ? '#333' : '#fff' }}>
      <h1>{title}</h1>
      <button onClick={toggleTheme}>Toggle Theme</button>
    </div>
  );
}

const EnhancedComponent = withTheme(BaseComponent);
// Usage: <EnhancedComponent title="Hello" />
```

**Explanation:**
- `ComponentType<T>` is the proper type for React components (includes static properties)
- `forwardRef` allows refs to pass through HOCs
- `displayName` helps with debugging in React DevTools
- `Omit<T, keyof InjectedProps>` removes injected props from the external API

---

## 32.9 Render Props Pattern

The render props pattern passes a function as a child or prop that returns JSX, allowing dynamic composition.

### Typing Render Props

```typescript
import { ReactNode } from 'react';

// Mouse position tracker with render prop
interface MousePosition {
  x: number;
  y: number;
}

interface MouseTrackerProps {
  render: (position: MousePosition) => ReactNode;
}

function MouseTracker({ render }: MouseTrackerProps) {
  const [position, setPosition] = useState<MousePosition>({ x: 0, y: 0 });

  const handleMouseMove = (event: React.MouseEvent<HTMLDivElement>) => {
    setPosition({
      x: event.clientX,
      y: event.clientY
    });
  };

  return (
    <div 
      style={{ height: '100vh' }} 
      onMouseMove={handleMouseMove}
    >
      {render(position)}
    </div>
  );
}

// Usage
function App() {
  return (
    <MouseTracker render={({ x, y }) => (
      <p>Mouse position: {x}, {y}</p>
    )} />
  );
}
```

### Generic Render Props with Data Fetching

```typescript
interface DataFetcherProps<T> {
  url: string;
  render: (state: {
    data: T | null;
    isLoading: boolean;
    error: Error | null;
    refetch: () => void;
  }) => ReactNode;
}

function DataFetcher<T>({ url, render }: DataFetcherProps<T>) {
  const [data, setData] = useState<T | null>(null);
  const [isLoading, setIsLoading] = useState(false);
  const [error, setError] = useState<Error | null>(null);

  const fetchData = useCallback(async () => {
    setIsLoading(true);
    setError(null);
    
    try {
      const response = await fetch(url);
      const json = await response.json() as T;
      setData(json);
    } catch (err) {
      setError(err instanceof Error ? err : new Error('Failed to fetch'));
    } finally {
      setIsLoading(false);
    }
  }, [url]);

  useEffect(() => {
    fetchData();
  }, [fetchData]);

  return <>{render({ data, isLoading, error, refetch: fetchData })}</>;
}

// Usage with specific type
interface User {
  id: number;
  name: string;
}

function UserProfile({ userId }: { userId: number }) {
  return (
    <DataFetcher<User> 
      url={`/api/users/${userId}`}
      render={({ data, isLoading, error }) => {
        if (isLoading) return <div>Loading...</div>;
        if (error) return <div>Error: {error.message}</div>;
        if (!data) return <div>No user</div>;
        
        return (
          <div>
            <h1>{data.name}</h1>
            <p>ID: {data.id}</p>
          </div>
        );
      }}
    />
  );
}
```

### Children as Function Pattern

```typescript
interface ToggleProps {
  children: (state: {
    on: boolean;
    toggle: () => void;
    setOn: (value: boolean) => void;
  }) => ReactNode;
  initialOn?: boolean;
}

function Toggle({ children, initialOn = false }: ToggleProps) {
  const [on, setOn] = useState(initialOn);

  const toggle = () => setOn(!on);

  return children({ on, toggle, setOn });
}

// Usage
function App() {
  return (
    <Toggle>
      {({ on, toggle }) => (
        <div>
          <button onClick={toggle}>
            {on ? 'ON' : 'OFF'}
          </button>
          {on && <p>The toggle is on!</p>}
        </div>
      )}
    </Toggle>
  );
}
```

---

## 32.10 Chapter Summary and Exercises

### Chapter Summary

This chapter covered integrating TypeScript with React:

**Key Concepts:**

1. **Setup**: Vite provides the best TypeScript + React experience with minimal configuration. Proper `tsconfig.json` settings ensure strict type checking.

2. **Component Props**: Use interfaces to define props, making components self-documenting and type-safe. `ReactNode` is the most flexible type for children.

3. **State Management**: `useState` can infer types or accept explicit generics. Use `useReducer` for complex state with discriminated unions for actions.

4. **Hooks**: 
   - `useRef` requires explicit types for DOM elements (e.g., `HTMLInputElement`)
   - `useCallback` and `useMemo` preserve function and value types
   - Custom hooks should be generic when working with varied data types

5. **Events**: React's synthetic events have specific types (`MouseEvent`, `ChangeEvent`, etc.) that provide access to typed event targets.

6. **Context**: Create context with `undefined` as default, then use a custom hook that throws if context is undefined. This ensures type safety.

7. **Advanced Patterns**:
   - HOCs use `ComponentType` and `forwardRef` for proper typing
   - Render props use generics to maintain type safety through the render function
   - Composition patterns work seamlessly with TypeScript's type inference

### Practical Exercises

**Exercise 1: Typed Component Library**

Create a type-safe Button component with variants:
- Props: `variant` ('primary' | 'secondary' | 'ghost'), `size` ('sm' | 'md' | 'lg'), `isLoading` (boolean), `leftIcon`/`rightIcon` (ReactNode)
- Use discriminated unions for different prop combinations (e.g., `href` for link buttons vs `onClick` for regular buttons)
- Ensure only valid prop combinations are allowed (can't have both `href` and `onClick`)

**Exercise 2: Generic Data Table**

Build a `DataTable<T>` component:
- Accepts `data: T[]`, `columns: ColumnDef<T>[]` where `ColumnDef` has `key: keyof T`, `header: string`, `render?: (row: T) => ReactNode`
- Implements sorting by column with typed sort functions
- Add row selection with `selectedRows: T['id'][]`
- Ensure type safety when accessing row properties

**Exercise 3: Form Hook**

Create `useForm<T>` hook:
- Accepts `initialValues: T` and `validate: (values: T) => Partial<Record<keyof T, string>>`
- Returns `{ values: T, errors: Partial<T>, touched: Partial<Record<keyof T, boolean>>, handleChange, handleSubmit, setFieldValue }`
- Ensure `handleChange` works with different input types (text, checkbox, select)
- Implement type-safe field names using `keyof T`

**Exercise 4: Context with Async Operations**

Build an AuthContext with:
- State: `user: User | null`, `isLoading: boolean`, `error: AuthError | null`
- Methods: `login(credentials)`, `logout()`, `register(data)`, `updateProfile(data)` - all returning promises
- Use discriminated unions for auth errors (NetworkError, ValidationError, UnauthorizedError)
- Implement error boundaries specific to auth errors

**Exercise 5: Compound Components**

Create a type-safe `Tabs` compound component:
- `Tabs` (container), `TabList`, `Tab`, `TabPanels`, `TabPanel`
- Use Context to share state between compound components
- Ensure `Tab` and `TabPanel` are only used within `Tabs`
- Support controlled and uncontrolled modes with proper typing

**Exercise 6: Higher-Order Component**

Build `withErrorBoundary` HOC:
- Wraps components to catch errors
- Injects `error: Error | null` and `resetError: () => void` props
- Properly forward refs and maintain display names
- Allow customization of fallback UI via props

**Exercise 7: Custom Hook with Generics**

Implement `usePagination<T>`:
- Accepts `items: T[]`, `pageSize: number`
- Returns `{ currentPage: number, totalPages: number, currentItems: T[], goToPage, nextPage, prevPage, hasNext, hasPrev }`
- Ensure it works with any array type
- Add option for server-side pagination with `totalCount: number`

**Exercise 8: Type-Safe Router**

Create a type-safe router wrapper around React Router:
- Define route config with path params (e.g., `/users/:id`)
- Generate typed `Link` components that validate path params
- Create `useParams` hook that returns typed params based on current route
- Implement `navigate` function with typed path arguments

### Additional Resources

- **React TypeScript Cheatsheet**: https://react-typescript-cheatsheet.netlify.app/
- **Total TypeScript** by Matt Pocock: Advanced React + TypeScript patterns
- **React Documentation**: https://react.dev/ (includes TypeScript examples)
- **TypeScript Handbook - JSX**: https://www.typescriptlang.org/docs/handbook/jsx.html

---

## Coming Up Next: Chapter 33 - TypeScript with Node.js

In the next chapter, we will explore using TypeScript on the server side with Node.js:

- Setting up Node.js projects with TypeScript
- Typing Express applications (Request, Response, Middleware)
- Typing Fastify and other modern frameworks
- NestJS architecture and dependency injection
- Database integration types (Prisma, TypeORM, Drizzle)
- Environment variables type safety
- Building and deploying TypeScript Node.js applications
- Testing Node.js TypeScript code

Understanding TypeScript with Node.js enables you to build type-safe full-stack applications with shared types between frontend and backend, improving code reuse and API contract reliability.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../8. advanced_patterns_and_techniques/31. advanced_design_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='33. typescript_with_nodejs.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
